## 读取数据集

Torchtext 中包含了上面我们要使用的 IMDB 数据集，并且还有读取语料库、词转词向量、词转下标、建立相应迭代器等功能，可以满足我们对文本的处理需求。

torchtext.datasets

这个过期了

现在使用 AutoTokenizer 这种类似的 HuggingFace datasets 读取 IMDB 数据集


In [8]:
# conda install -n pytorch311 -c pytorch torchtext

In [18]:
import torch, torchvision, torchaudio
print(torch.__version__)       # 2.5.1+cu121
print(torchvision.__version__) # 0.20.1+cu121
print(torch.cuda.is_available())  # True(如果驱动OK)



2.5.1+cu121
0.20.1+cu121
True


LSTM 是一个“时间序列模型”，它要“顺序地读”每个单词的向量。

每个时间步输入一个词向量，累计上下文，最终得到整句话的语义。

文本 → token id → [B, T] → embedding lookup → [B, T, E]

| 阶段       | 数据形式          | 维度      | 示例                                    |
| -------- | ------------- | ------- | ------------------------------------- |
| 原始输入     | 字符串           | —       | `"This movie is amazing!"`            |
| 分词       | list[str]     | [T]     | `["this","movie","is","amazing","!"]` |
| 映射索引     | list[int]     | [T]     | `[2,3,4,5,6]`                         |
| Padding  | list[int]     | [T=8]   | `[2,3,4,5,6,0,0,0]`                   |
| Batch 合并 | tensor[int]   | [B,T]   | `torch.Size([3,8])`                   |
| 词嵌入      | tensor[float] | [B,T,E] | `torch.Size([3,8,300])`               |


### LSTM 结构

普通 RNN 结构如下

$$
h_t=\tanh \left(W_{x h} x_t+W_{h h} h_{t-1}+b\right)
$$

但如果句子太长（几十或上百词），梯度会消失或爆炸，导致前面信息难以保留。

STM 在每个时间步维护两种状态：

hₜ：隐藏状态（短期记忆，用于输出）

cₜ：细胞状态（长期记忆，用于信息保留）

它通过“门”来控制信息的流入、遗忘、输出。

![image16.png](mdfiles/image16.png)

LSTM 在分割数据的时候如何处理呢？

先打乱数据还是先切割成cell再打乱？如果这个顺序不对会有什么影响？

一般是先打乱数据，再按 batch 切割成 cell。这样可以确保每个 batch 的数据都是随机的，避免模型学习到数据的顺序信息。

我没懂如果先打乱数据再切割成cell会避免模型学习到数据的顺序信息？LSTM本身是有顺序性的，如果打乱数据，LSTM的顺序性还存在吗？

LSTM 的核心设计是

学习输入序列中 时间步之间的依赖关系

LSTM 关心的是单个样本（sequence）内部的顺序

对于鞋垫的连续采集来说，我们还是需要保持单个样本内部的时间顺序

“先在时间点层面打乱，再切窗口”，那每个窗口内部的时间步将来自乱序的全局时间，LSTM 再强也学不到真实的动力学模式

1. 先按时间顺序切窗口（或滑窗生成序列）
2. 再在序列层面 shuffle 形成 batch 防止序列间的依赖 （但是存在例外：见下文）


什么时候不要（或少）打乱？

有两类常见例外：

1. 语言建模 / 长序列建模（TBPTT）

你把超长序列按时间顺序切成多个连续小块（如每块 256 步），

训练时把上一个小块的隐藏状态传给下一个小块（stateful 或 TBPTT），

因为依赖跨块存在，所以不能在块之间打乱，通常用 SequentialSampler 或自定义顺序迭代器。

2. 在线/实时预测需要平滑状态

评估/部署时要按时间顺序输入，可能需要保留隐藏状态。


训练/验证划分（防止泄漏） 这个部分是先划分出训练集和验证集，然后在各自集内打乱形成 batch。

按会话/时间段划分（而不是随机按窗口划分）：

例如：前 70% 时间段做训练，后 30% 做验证/测试； 所以按照时间窗口先划分训练集和验证集

或者按不同采集 session划分，保证验证集中窗口不与训练集窗口发生重叠。

---

TBPTT（Truncated Backpropagation Through Time）截断时间反向传播

TBPTT 必须保持 cell（即序列块）之间的时间顺序。
因为它要在块之间连续传递 LSTM 的隐藏状态 (h, c)，以保留长时间依赖。

| 训练方式                      | 每个 batch 的序列      | 是否传递隐藏状态       | 能否乱序（shuffle） | 典型用途           |
| ------------------------- | ----------------- | -------------- | ------------- | -------------- |
| **Stateless LSTM**        | 独立样本（每段序列是完整样本）   | ❌ 每次都 `h=c=0`  | ✅ 可以随意打乱      | 语音段分类、传感器动作识别等 |
| **Stateful / TBPTT LSTM** | 从同一长序列切块（例如按时间滑动） | ✅ 块之间传 `(h,c)` | ❌ 必须保持时间顺序    | 语言建模、连续信号预测等   |


---


| 场景       | Block长度（步）         | 截断策略    | 是否保状态 | 备注          |
| -------- | ------------------ | ------- | ----- | ----------- |
| 语言建模     | 100–256 token      | 每块反传一次  | 是     | 经典 RNN LM   |
| 语音信号     | 200–800 帧          | 每块反传    | 是     | 约 0.2–0.5 s |
| EEG/鞋垫信号 | 160–320 帧          | 每块反传    | 是     | 约 2–4 s     |
| RL 环境    | 20–40 step rollout | 每 n 步更新 | 是     | 平衡稳定性与效率    |
| 视频流      | 16–64 帧            | 每块反传    | 是     | clip 连续预测   |


这类场景就是你（鞋垫、肌电、BioZ）非常相关的方向。

任务：

步态相位识别 / 足底压力时间序列预测

肌电信号分类 / 脑电识别

连续生理信号（血压、心率）估计

问题：

数据连续长（几十秒、几分钟）

行为状态连续变化，不宜打乱

TBPTT 的优势：

能让模型保持跨步态的时间关联；

避免每窗口独立训练的短时割裂；

同时避免全序列反传显存爆。

📈 举例：鞋垫 80 Hz × 60 s → 4800 步。
TBPTT 可设 block=160 步（2 s），在块间保留状态，反向传播只回溯最近 160 步。


#### Stateless（最常用，序列彼此独立）

假设：鞋垫 80 Hz，时长 10 min = 600 s → 48,000 步。
选择：窗口长度 T=160（≈2 s），步幅/重叠 S=80（1 s 步幅，50% 重叠）。

先按时间顺序滑窗生成样本：
样本数 ≈ ((48000 - 160) / 80) + 1 = 599 段。

再在“样本层级”shuffle，组成 batch。

batch size：常见 32/64/128；例如 B=64。

每个 batch 是 [B, T, F]，T=160，F=特征维度（传感器通道数）。

隐状态：每个样本独立做预测，h0,c0=0；不在样本之间传状态。

优点：实现简单、并行度高。
注意：验证/测试阶段按时间顺序评估（shuffle=False）。

适用：动作/阶段分类、短窗回归、不需要跨窗上下文的任务。


#### TBPTT / Stateful（需要跨窗上下文）

目标：保持长时记忆但只对最近 N 步反传。

两种常见批处理方式

方式 B1｜单流（batch=1 或小 batch）

仍是 48,000 步；设 block（截断长度）N=160。

按时间顺序依次取：

Block1：1–160

Block2：161–320

… 直到结束

块间保留隐藏状态 (h,c)；每块反传后 h,c = detach()。

不能打乱块顺序。

方式 B2｜多流并行（推荐）

先把一条长序列**等分成 B 条“子流”**并行（提升吞吐）：

例如设 B=8，每条子流长度 48000/8 = 6000 步。

训练时“同步推进”每条子流的下一个 block：

step 1：取每条子流的 block1（共 8 个块）→ 组成一个 batch（形状 [8, 160, F]）

step 2：再取每条子流的 block2 → 下一个 batch

…

每条子流内部块间传 (h,c)；子流之间互不影响（各自的 (h,c) 独立）。

每条子流可在开始时用随机起点，但子流内部严禁打乱。

数量直觉：每子流 6000/160 ≈ 37 个整块（余 80 步可忽略或 padding），每个“时间步”你都会处理一个含 8 个块的 batch。

### LSTM 后面的linear层 

目的：把 LSTM 的输出特征映射到我们需要的类别数上。

LSTM 输出一个 600 维特征（双向隐藏层各 300）

我们希望做二分类（正负情感）。

``` python
fc_in = 600
output_dim = 2
self.fc = nn.Linear(600, 2)
```
这 [B, 2] 的两个数字就表示：

第0维：负面分数

第1维：正面分数
``` python
fc_in = 600
output_dim = 2
self.fc = nn.Linear(600, 2)
```
这 [B, 2] 的两个数字就表示：

第0维：负面分数

第1维：正面分数



### Dropout 层

Dropout 是一种 正则化（Regularization） 方法，目的是防止过拟合。

Embedding后（输入层）

emb = self.dropout(self.embedding(ids))

全连接层前（输出层）

feat = self.dropout(feat)

保证期望不变，但每次丢掉一部分神经元，让模型更健壮。

$$
y_i= \begin{cases}0 & \text { with prob } p \\ x_i /(1-p) & \text { with prob }(1-p)\end{cases}
$$

## 模型细节

最后时刻的隐藏态
$$
\text { feat }= \begin{cases}h_{\vec{L}, T} \oplus h_{\overleftarrow{L}, T}^{\leftarrow} & \text { (双向) } \\ h_{L, T} & \text { (单向) }\end{cases}
$$

全句平均和全句最强
$$
\text { feat }=\left(\operatorname{mean}_{t \leq \ell_b} h_{L, t}\right) \oplus\left(\max _{t \leq \ell_b} h_{L, t}\right)
$$

| 对比项              | **旧版：最后层最后隐藏态 (`h_n`)**                                                     | **新版：全序列 Mean + Max Pooling**                                                 |
| :--------------- | :-------------------------------------------------------------------------- | :---------------------------------------------------------------------------- |
| **特征来源**         | 取 LSTM 最后一层的最后时间步 hidden：<br>`feat = h_n[-1]` 或拼接双向的 `h_n[-2], h_n[-1]`     | 对所有时间步的输出 `out` 做池化：<br>`feat = [mean_pool, max_pool]`                        |
| **计算公式**         | 单点特征：<br>$$h_{feat} = h_{L, T}^{\rightarrow} \oplus h_{L, T}^{\leftarrow}$$ | 全句聚合：<br>$$h_{feat} = (\text{mean}*t h*{L,t}) \oplus (\text{max}*t h*{L,t})$$ |
| **维度大小**         | `[B, H]`（单向）或 `[B, 2H]`（双向）                                                 | `[B, 2C]`，其中 `C = H` 或 `2H`（双向再×2）                                            |
| **`fc` 输入维度**    | `fc_in = hidden_dim * (2 if bidirectional else 1)`                          | `fc_in = 2 * hidden_dim * (2 if bidirectional else 1)`                        |
| **是否依赖最后时刻**     | ✅ 仅依赖句末隐藏状态                                                                 | ❌ 聚合整个时间序列                                                                    |
| **捕捉长句信息**       | 较弱（长句后部主导）                                                                  | 强（全局平均+最显著特征）                                                                 |
| **处理否定/转折句**     | 容易误判（“not good”→positive）                                                   | 稳定得多（“not good”中“not”步仍参与 mean/max）                                           |
| **实现复杂度**        | 简单：直接取 `h_n`                                                                | 略复杂：需构造 `mask` 并计算池化                                                          |
| **运行速度**         | 稍快（少一步池化）                                                                   | 稍慢（多一次全序列操作，差异可忽略）                                                            |
| **对句长敏感度**       | 高（过长句子，早期信息难保留）                                                             | 低（均值聚合可平衡全句信息）                                                                |
| **数值稳定性处理**      | 不需要 mask                                                                    | 需处理 pad 部分（mask + `-inf` + clamp）                                             |
| **模型容量**         | 参数少                                                                         | 参数略多（`fc` 输入翻倍）                                                               |
| **泛化能力**         | 容易过拟合训练分布                                                                   | 更稳健、泛化更好                                                                      |
| **训练收敛速度**       | 快                                                                           | 稍慢（梯度来自更多时间步）                                                                 |
| **适合任务**         | 短句、结构简单、语义集中任务                                                              | 长句、否定句、语法复杂任务（IMDB类情感分析）                                                      |
| **实现示例**         | `feat = h_n[-1]` 或拼接双向                                                      | `feat = torch.cat([mean_pool, max_pool], dim=1)`                              |
| **Dropout 作用位置** | embedding 后 + fc 前                                                          | embedding 后 + pooling 后（同）                                                    |


## 结果分析

我现在对于转折等等，以及一些复杂的双重否定等等的句子，模型还是不能很好地捕捉到情感倾向。
而且对于一些中性的句子，模型也不能很好地判断出来。

解决方案：
1. 增加训练轮数，比如从10轮增加到30轮，观察验证集的准确率是否提升。
2. 尝试更复杂的模型结构，比如增加LSTM层数或隐藏单元数。
3. 引入注意力机制，让模型更好地关注句子中的关键部分。
4. 使用预训练的词向量（如GloVe或BERT）来初始化嵌入层，提升词语表示能力。